In [2]:
from openai import OpenAI
import time

class LegalLLaMA3_70B_Answerer:
    def __init__(self, api_key, model="meta/llama3-70b-instruct"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://integrate.api.nvidia.com/v1"
        )
        self.model = model

    def build_prompt(self, query, context_docs):
        context = "\n\n".join([f"Context {i+1}:\n{doc.strip()}" for i, doc in enumerate(context_docs)])
        prompt = (
            f"{context}\n\n"
            f"Question: {query.strip()}\n\n"
            f"Answer with a complete response followed by a short reasoning.\n"
            f"Format:\nAnswer: <your answer>\nReasoning: <your reasoning>"
        )
        return prompt

    def generate(self, query, context_docs, debug=False, retries=3, wait_time=10):
        prompt = self.build_prompt(query, context_docs)
        messages = [{"role": "user", "content": prompt}]
        
        for attempt in range(retries):
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=0.3,
                    top_p=1.0,
                    max_tokens=1024,
                    stream=False
                )
                response_text = completion.choices[0].message.content.strip()

                if debug:
                    print("🧠 Raw Response:\n", response_text)

                # Parse output
                answer = ""
                reasoning = ""
                if "Answer:" in response_text:
                    answer_part = response_text.split("Answer:", 1)[-1]
                    if "Reasoning:" in answer_part:
                        answer, reasoning = answer_part.split("Reasoning:", 1)
                    else:
                        answer = answer_part
                else:
                    answer = response_text

                return {
                    "answer": answer.strip(),
                    "reasoning": reasoning.strip() if reasoning else "(No explicit reasoning found)"
                }

            except Exception as e:
                print(f"⚠️ Attempt {attempt+1} failed: {e}")
                if attempt < retries - 1:
                    print(f"⏳ Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise RuntimeError("❌ All retries failed for LLaMA3-70B")

In [4]:
query = "Is consent required for a police officer to search a person's bag during a routine traffic stop?"
context_docs = [
    "The Fourth Amendment protects against unreasonable searches and seizures.",
    "Consent is one of the exceptions to the warrant requirement under the Fourth Amendment.",
    "If a person voluntarily agrees to a search, the search is considered lawful.",
    "However, consent must be given freely and not coerced by the police officer."
]

llm = LegalLLaMA3_70B_Answerer(api_key="token_here")
response = llm.generate(query, context_docs, debug=True)

print("\n✅ Answer:", response["answer"])
print("📚 Reasoning:", response["reasoning"])

🧠 Raw Response:
 Answer: No, consent is not required, but it must be voluntarily given if obtained.

Reasoning: According to Context 2 and 3, consent is an exception to the warrant requirement, and if a person voluntarily agrees to a search, it is considered lawful. However, as stated in Context 4, the consent must be given freely and not coerced by the police officer. This implies that the police officer may not need consent to search a person's bag during a routine traffic stop, but if they do obtain consent, it must be voluntary and not coerced.

✅ Answer: No, consent is not required, but it must be voluntarily given if obtained.
📚 Reasoning: According to Context 2 and 3, consent is an exception to the warrant requirement, and if a person voluntarily agrees to a search, it is considered lawful. However, as stated in Context 4, the consent must be given freely and not coerced by the police officer. This implies that the police officer may not need consent to search a person's bag dur